<a href="https://colab.research.google.com/github/LucasFDutra/machine_learning_studies/blob/master/machine_learning_studies/Music_Generation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libs


In [22]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
assert len(tf.config.list_physical_devices('GPU')) > 0

# Dataset

Vou pegar os dados com o auxilio da biblioteca do MIT


In [23]:
songs = mdl.lab1.load_training_data()

Found 816 songs in text


In [24]:
songs[0]

'X:2\nT:An Buachaill Dreoite\nZ: id:dc-hornpipe-2\nM:C|\nL:1/8\nK:G Major\nGF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!\nDGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!\nGA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!\nB2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!'

# Criando um vocabulário

Verificar os caracteres únicos

In [25]:
songs_joined = "\n\n".join(songs)

vocab = sorted(set(songs_joined))

print ('Existem ', len(vocab), ' caracteres diferentes no vocabulário')

Existem  83  caracteres diferentes no vocabulário


# Criando um dicionário de dados

Vou pegar todos os carateres presentes no dataset e atribuir um valor numérico para cada um.

ex: a = 0, b = 1, ...

E também criarei um meio de voltarmos, colocando os caracteres em um array

ex: `['a', 'b']`

Assim saberemos que o caractere representado pelo número 0 é o 'a', pois ele ocupa a posição zero no array

In [26]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

print('em char2idx: ', list(char2idx.items())[0])

em char2idx:  ('\n', 0)


# Convertendo o array de letras nos numeros correspondentes do nosso diconário de dados

In [27]:
def vectorize_string(string):
    vectorized_output = np.array([char2idx[char] for char in string])
    return vectorized_output

vectorized_songs = vectorize_string(songs_joined)

vectorized_songs

array([49, 22, 14, ..., 22, 82,  2])

# Definir função para pegar batchs de dados
```python
vectorized_songs = [1,2,3,4]

seq_length = 3

x = [1,2,3]
y = [2,3,4]
```

In [28]:
def get_batch(vectorized_songs, seq_length, batch_size):
    n = len(vectorized_songs) - 1
    idx = np.random.choice(n - seq_length, batch_size)

    input_batch = [vectorized_songs[i : seq_length+i] for i in idx]
    output_batch = [vectorized_songs[i+1 : seq_length+i+1] for i in idx]

    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])

    return x_batch, y_batch

In [29]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

print(x_batch, y_batch)

[[60 59 60 82  2]] [[59 60 82  2  0]]


# Construindo o modelo

uma camada `tf.keras.layers.Embedding`

uma camada `tf.keras.layers.LSTM`

uma camada `tf.keras.layers.Dense

In [30]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, recurrent_activation='sigmoid', return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (32, None, 256)           21248     
_________________________________________________________________
lstm_2 (LSTM)                (32, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (32, None, 83)            85075     
Total params: 5,353,299
Trainable params: 5,353,299
Non-trainable params: 0
_________________________________________________________________


In [31]:
x, y = get_batch(vectorized_songs=vectorized_songs, seq_length=100, batch_size=32)
pred = model(x)
pred

<tf.Tensor: shape=(32, 100, 83), dtype=float32, numpy=
array([[[ 2.34618690e-03,  1.42713194e-03, -6.00618427e-04, ...,
          7.28861894e-04, -2.77000340e-03, -5.20943524e-03],
        [ 7.36461952e-03,  1.68872508e-03,  1.41698902e-03, ...,
          1.17855787e-04, -2.01251218e-03, -3.54254525e-03],
        [ 8.05306714e-03,  2.15858640e-03,  4.02011792e-04, ...,
          6.15835423e-04, -4.05140594e-03, -8.82963743e-03],
        ...,
        [ 1.42867975e-02, -1.79493672e-03, -4.65434697e-03, ...,
          9.79622640e-03,  1.68226846e-02,  9.79590509e-03],
        [ 1.10429861e-02, -3.86076933e-03, -8.94935150e-03, ...,
          7.51943793e-03,  5.17214881e-03,  4.76680184e-03],
        [ 1.67431124e-02,  2.60829669e-03, -5.55738714e-03, ...,
          8.12738482e-03,  2.94378609e-03,  6.06860220e-03]],

       [[ 4.31632111e-03,  3.79699282e-03, -4.06862749e-03, ...,
         -5.98797295e-03,  6.60364982e-03, -8.62081069e-04],
        [ 4.15931456e-03, -1.01192854e-05, -7.05

In [37]:
pred[0].shape

TensorShape([100, 83])

In [41]:
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([80, 14, 48,  3, 68, 14, 68, 61, 45, 40, 26,  0, 75, 74, 74, 60, 48,
       31, 30, 15, 63, 37, 21, 30, 27, 10, 13, 13, 12, 59, 37, 17, 55, 19,
        7,  7, 40, 15,  4, 12, 16, 15, 41, 20, 27, 22, 60, 15, 76, 16, 63,
       74, 71, 52, 25, 30,  3, 36,  1, 41, 40, 21, 69,  7, 63, 72, 61, 31,
       24, 40, 70, 67, 82, 67, 11, 47,  8, 37, 25, 80, 12, 47, 19, 21, 12,
       15, 55,  5, 35, 20, 77, 17, 66, 57, 56, 65, 69, 21, 63, 24])

# Calculando a perda

In [121]:
def compute_loss(y_true, y_pred):
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
    return loss

example_batch_loss = compute_loss(y, pred)
example_batch_loss

<tf.Tensor: shape=(32, 100), dtype=float32, numpy=
array([[4.417932 , 4.4255557, 4.4291143, ..., 4.425583 , 4.4246116,
        4.4168687],
       [4.424343 , 4.420055 , 4.4208255, ..., 4.4234843, 4.4082265,
        4.413118 ],
       [4.415841 , 4.4199114, 4.421339 , ..., 4.426486 , 4.417754 ,
        4.413988 ],
       ...,
       [4.4255066, 4.421087 , 4.4281716, ..., 4.412753 , 4.435701 ,
        4.419826 ],
       [4.4198995, 4.4246593, 4.413592 , ..., 4.409494 , 4.4317822,
        4.4178486],
       [4.4200954, 4.423635 , 4.4131417, ..., 4.416474 , 4.4195123,
        4.4114084]], dtype=float32)>

In [122]:
example_batch_loss.numpy().mean()

4.41927